# DX 704 Week 9 Project

This week's project will build an email spam classifier based on the Enron email data set.
You will perform your own feature extraction, and use naive Bayes to estimate the probability that a particular email is spam or not.
Finally, you will review the tradeoffs from different thresholds for automatically sending emails to the junk folder.

The full project description and a template notebook are available on GitHub: [Project 9 Materials](https://github.com/bu-cds-dx704/dx704-project-09).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Download Data Set

We will be using the Enron spam data set as prepared in this GitHub repository.

https://github.com/MWiechmann/enron_spam_data

You may need to download this differently depending on your environment.

In [2]:
!wget https://github.com/MWiechmann/enron_spam_data/raw/refs/heads/master/enron_spam_data.zip

--2025-11-03 06:39:32--  https://github.com/MWiechmann/enron_spam_data/raw/refs/heads/master/enron_spam_data.zip
Resolving github.com (github.com)... 140.82.112.3
Connecting to github.com (github.com)|140.82.112.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/MWiechmann/enron_spam_data/refs/heads/master/enron_spam_data.zip [following]
--2025-11-03 06:39:32--  https://raw.githubusercontent.com/MWiechmann/enron_spam_data/refs/heads/master/enron_spam_data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15642124 (15M) [application/zip]
Saving to: ‘enron_spam_data.zip.18’

enron_spam_data.zip 100%[===================>]  14.92M  77.3MB/s    in 0.2s    

2025-11-03 06:39:32 (77.3 MB/s) - ‘enron_

In [3]:
import pandas as pd

In [4]:
# pandas can read the zip file directly
enron_spam_data = pd.read_csv("enron_spam_data.zip")
enron_spam_data

,Message ID,Subject,Message,Spam/Ham,Date
0,0,christmas tree farm pictures,NaN,ham,1999-12-10
1,1,"vastar resources , inc .","gary , production from the high island larger ...",ham,1999-12-13
2,2,calpine daily gas nomination,- calpine daily gas nomination 1 . doc,ham,1999-12-14
3,3,re : issue,fyi - see note below - already done .\nstella\...,ham,1999-12-14
4,4,meter 7268 nov allocation,fyi .\n- - - - - - - - - - - - - - - - - - - -...,ham,1999-12-14
...,...,...,...,...,...
33711,33711,= ? iso - 8859 - 1 ? q ? good _ news _ c = eda...,"hello , welcome to gigapharm onlinne shop .\np...",spam,2005-07-29
33712,33712,all prescript medicines are on special . to be...,i got it earlier than expected and it was wrap...,spam,2005-07-29
33713,33713,the next generation online pharmacy .,are you ready to rock on ? let the man in you ...,spam,2005-07-30
33714,33714,bloow in 5 - 10 times the time,learn how to last 5 - 10 times longer in\nbed ...,spam,2005-07-30


In [5]:
(enron_spam_data["Spam/Ham"] == "spam").mean()

np.float64(0.5092834262664611)

## Part 2: Design a Feature Extractor

Design a feature extractor for this data set and write out two files of features based on the text.
Don't forget that both the Subject and Message columns are relevant sources of text data.
For each email, you should count the number of repetitions of each feature present.
The auto-grader will assume that you are using a multinomial distribution in the following problems.

In [6]:
# YOUR CHANGES HERE

import pandas as pd
import json
import re
from collections import Counter

_word_re = re.compile(r"[A-Za-z]{2,}")

def tokenize(text: str):
    if not isinstance(text, str):
        return []
    return [w.lower() for w in _word_re.findall(text)]

def featurize_row(subject, message) -> dict:
    subject = subject if isinstance(subject, str) else ""
    message = message if isinstance(message, str) else ""

    toks = tokenize(subject) + tokenize(message)
    feats = Counter(toks)

    blob = subject + " " + message
    feats["__has_http__"]   = int("http" in blob.lower())
    feats["__has_dollar__"] = int("$" in blob)
    feats["__subject_len__"] = len(subject) // 20

    #  only positive counts
    return {k: int(v) for k, v in feats.items() if v > 0}

rows = []
for _, r in enron_spam_data.iterrows():
    mid = int(r["Message ID"])
    feats = featurize_row(r.get("Subject", ""), r.get("Message", ""))
    rows.append({
        "Message ID": mid,
        "features_json": json.dumps(feats, separators=(",", ":"))
    })

feat_df = pd.DataFrame(rows)



Assign a row to the test data set if `Message ID % 30 == 0` and assign it to the training data set otherwise.
Write two files, "train-features.tsv" and "test-features.tsv" with two columns, Message ID and features_json.
The features_json column should contain a JSON dictionary where the keys are your feature names and the values are integer feature values.
This will give us a sparse feature representation.


In [7]:
# YOUR CHANGES HERE

test_mask = feat_df["Message ID"] % 30 == 0
train_features = feat_df.loc[~test_mask].copy()
test_features  = feat_df.loc[test_mask].copy()

train_features.to_csv("train-features.tsv", sep="\t", index=False)
test_features.to_csv("test-features.tsv", sep="\t", index=False)

print("Saved train-features.tsv:", train_features.shape)
print(" Saved test-features.tsv:", test_features.shape)
train_features.head()


Saved train-features.tsv: (32592, 2)
 Saved test-features.tsv: (1124, 2)


,Message ID,features_json
1,1,"{""vastar"":6,""resources"":4,""inc"":4,""gary"":2,""pr..."
2,2,"{""calpine"":2,""daily"":2,""gas"":2,""nomination"":2,..."
3,3,"{""re"":2,""issue"":4,""fyi"":1,""see"":1,""note"":1,""be..."
4,4,"{""meter"":3,""nov"":3,""allocation"":3,""fyi"":1,""for..."
5,5,"{""mcmullen"":2,""gas"":5,""for"":2,""jackie"":1,""sinc..."


Submit "train-features.tsv" and "test-features.tsv" in Gradescope.

Hint: these features will be graded based on the test accuracy of a logistic regression based on the training features.
This is to make sure that your feature set is not degenerate; you do not need to compute this regression yourself.
You can separately assess your feature quality based on your results in part 6.

## Part 3: Compute Conditional Probabilities

Based on your training data, compute appropriate conditional probabilities for use with naïve Bayes.
Use of additive smoothing with $\alpha=1$ to avoid zeros.


In [8]:
# YOUR CHANGES HERE

import pandas as pd
import json
from collections import Counter, defaultdict

train_df = pd.read_csv("train-features.tsv", sep="\t")
train_labels = enron_spam_data[["Message ID", "Spam/Ham"]].copy()
train_data = train_df.merge(train_labels, on="Message ID")

word_counts = {
    "spam": Counter(),
    "ham": Counter(),
}
doc_counts = Counter({"spam": 0, "ham": 0})

for _, row in train_data.iterrows():
    feats = json.loads(row["features_json"])
    label = "spam" if row["Spam/Ham"].strip().lower() == "spam" else "ham"
    doc_counts[label] += 1
    word_counts[label].update(feats)

vocab = set(word_counts["spam"]) | set(word_counts["ham"])
vocab_size = len(vocab)

feature_probs = []
for word in vocab:
    spam_ct = word_counts["spam"][word]
    ham_ct = word_counts["ham"][word]

    spam_prob = (spam_ct + 1) / (sum(word_counts["spam"].values()) + vocab_size)
    ham_prob  = (ham_ct + 1) / (sum(word_counts["ham"].values()) + vocab_size)

    feature_probs.append({
        "feature": word,
        "ham_probability":  ham_prob,
        "spam_probability": spam_prob
    })



Save the conditional probabilities in a file "feature-probabilities.tsv" with columns feature, ham_probability and spam_probability.

In [9]:
# YOUR CHANGES HERE

probs_df = pd.DataFrame(feature_probs, columns=["feature","ham_probability","spam_probability"])
probs_df.to_csv("feature-probabilities.tsv", sep="\t", index=False)
print("Saved feature-probabilities.tsv with", len(probs_df), "features.")
probs_df.head(10)


Saved feature-probabilities.tsv with 141211 features.


,feature,ham_probability,spam_probability
0,busquedas,2.450672e-07,1.216742e-06
1,olivustea,2.450672e-07,1.825113e-06
2,windowprops,2.450672e-07,9.125567e-07
3,htmlherebr,2.450672e-07,6.083711e-07
4,comunicarlo,2.450672e-07,6.083711e-07
5,cqi,2.450672e-07,9.125567e-07
6,consolers,2.450672e-07,9.125567e-07
7,hoqhxg,2.450672e-07,6.083711e-07
8,puseyism,2.450672e-07,6.083711e-07
9,postalprovincia,2.450672e-07,6.083711e-07


Submit "feature-probabilities.tsv" in Gradescope.

## Part 4: Implement a Naïve Bayes Classifier

Implement a naïve Bayes classifier based on your previous feature probabilities.

In [10]:
# YOUR CHANGES HERE

import pandas as pd, numpy as np, json, math

train_feats = pd.read_csv("train-features.tsv", sep="\t")

feat_probs = pd.read_csv("feature-probabilities.tsv", sep="\t")

spam_col = "p_spam" if "p_spam" in feat_probs.columns else "spam_probability"
ham_col  = "p_ham"  if "p_ham"  in feat_probs.columns else "ham_probability"

p_spam = dict(zip(feat_probs["feature"], feat_probs[spam_col]))
p_ham  = dict(zip(feat_probs["feature"],  feat_probs[ham_col]))

EPS = 1e-12

train_ids = set(train_feats["Message ID"].tolist())
train_labels = enron_spam_data[enron_spam_data["Message ID"].isin(train_ids)]
prior_spam = (train_labels["Spam/Ham"].str.lower() == "spam").mean()
prior_ham  = 1.0 - prior_spam

log_prior_spam = math.log(max(prior_spam, EPS))
log_prior_ham  = math.log(max(prior_ham,  EPS))

def predict_proba_from_features(feats_json: str):

    feats = json.loads(feats_json)

    logp_h = log_prior_ham
    logp_s = log_prior_spam

    for tok, cnt in feats.items():
        ph = p_ham.get(tok,  EPS)
        ps = p_spam.get(tok, EPS)
        logp_h += cnt * math.log(ph)
        logp_s += cnt * math.log(ps)

    m = max(logp_h, logp_s)
    eh = math.exp(logp_h - m)
    es = math.exp(logp_s - m)
    z = eh + es
    return {"ham": eh / z, "spam": es / z}

rows = []
for _, r in train_feats.iterrows():
    probs = predict_proba_from_features(r["features_json"])
    rows.append({
        "Message ID": int(r["Message ID"]),
        "ham":  probs["ham"],
        "spam": probs["spam"],
    })






Save your prediction probabilities to "train-predictions.tsv" with columns Message ID, ham and spam.

In [11]:
# YOUR CHANGES HERE

train_pred_df = pd.DataFrame(rows)
train_pred_df.to_csv("train-predictions.tsv", sep="\t", index=False)
print(f"Saved train-predictions.tsv with {len(train_pred_df)} rows.")
train_pred_df.head()


Saved train-predictions.tsv with 32592 rows.


,Message ID,ham,spam
0,1,1.0,1.441650e-166
1,2,1.0,2.297535e-12
2,3,1.0,2.379096e-149
3,4,1.0,7.902319e-143
4,5,1.0,6.482699e-38


Submit "train-predictions.tsv" in Gradescope.

## Part 5: Predict Spam Probability for Test Data

Use your previous classifier to predict spam probability for the test data.

In [12]:
# YOUR CHANGES HERE

import pandas as pd, numpy as np, json, math

test_feats = pd.read_csv("test-features.tsv", sep="\t")

feat_probs = pd.read_csv("feature-probabilities.tsv", sep="\t")

spam_col = "p_spam" if "p_spam" in feat_probs.columns else "spam_probability"
ham_col  = "p_ham"  if "p_ham"  in feat_probs.columns else "ham_probability"

p_spam = dict(zip(feat_probs["feature"], feat_probs[spam_col]))
p_ham  = dict(zip(feat_probs["feature"],  feat_probs[ham_col]))

EPS = 1e-12  

train_ids = set(pd.read_csv("train-features.tsv", sep="\t")["Message ID"])
train_labels = enron_spam_data[enron_spam_data["Message ID"].isin(train_ids)]
prior_spam = (train_labels["Spam/Ham"].str.lower() == "spam").mean()
prior_ham  = 1.0 - prior_spam
log_prior_spam = math.log(max(prior_spam, EPS))
log_prior_ham  = math.log(max(prior_ham,  EPS))

def predict_proba_from_features(feats_json: str):
    feats = json.loads(feats_json)

    logp_h = log_prior_ham
    logp_s = log_prior_spam
    for tok, cnt in feats.items():
        ph = p_ham.get(tok,  EPS)
        ps = p_spam.get(tok, EPS)
        logp_h += cnt * math.log(ph)
        logp_s += cnt * math.log(ps)

    m = max(logp_h, logp_s)          
    eh, es = math.exp(logp_h - m), math.exp(logp_s - m)
    z = eh + es
    return {"ham": eh/z, "spam": es/z}

rows = []
for _, r in test_feats.iterrows():
    probs = predict_proba_from_features(r["features_json"])
    rows.append({
        "Message ID": int(r["Message ID"]),
        "ham":  probs["ham"],
        "spam": probs["spam"],
    })






Save your prediction probabilities in "test-predictions.tsv" with the same columns as "train-predictions.tsv".

In [13]:
# YOUR CHANGES HERE


pred_probs_df = pd.DataFrame(rows)
pred_probs_df.to_csv("test-predictions.tsv", sep="\t", index=False)
print(f"Saved test-predictions.tsv with {len(pred_probs_df)} rows.")
pred_probs_df.head()

Saved test-predictions.tsv with 1124 rows.


,Message ID,ham,spam
0,0,0.035748,9.642522e-01
1,30,1.000000,1.550662e-79
2,60,1.000000,2.505070e-12
3,90,1.000000,1.138224e-32
4,120,1.000000,4.227987e-169


Submit "test-predictions.tsv" in Gradescope.

## Part 6: Construct ROC Curve

For every probability threshold from 0.01 to .99 in increments of 0.01, compute the false and true positive rates from the test data using the spam class for positives.
That is, if the predicted spam probability is greater than or equal to the threshold, predict spam.

In [14]:
# YOUR CHANGES HERE

import pandas as pd
import numpy as np

preds = pd.read_csv("test-predictions.tsv", sep="\t")
if "spam" in preds.columns:
    prob_col = "spam"
elif "p_spam" in preds.columns:
    prob_col = "p_spam"
else:
    raise ValueError(f"Couldn't find spam prob column in predictions: {list(preds.columns)}")

truth = enron_spam_data[["Message ID", "Spam/Ham"]].copy()
truth["y_true"] = truth["Spam/Ham"].str.lower().map({"spam": 1, "ham": 0})
df = preds.merge(truth[["Message ID", "y_true"]], on="Message ID", how="inner")

assert df[prob_col].between(0,1).all(), "Predicted spam probs must be in [0,1]"
assert df["y_true"].isin([0,1]).all(), "Labels must be 0/1"

rows = []
for t in np.arange(0.01, 1.00, 0.01):
    y_pred = (df[prob_col] >= t).astype(int).to_numpy()
    y_true = df["y_true"].to_numpy()

    tp = ((y_pred == 1) & (y_true == 1)).sum()
    fp = ((y_pred == 1) & (y_true == 0)).sum()
    tn = ((y_pred == 0) & (y_true == 0)).sum()
    fn = ((y_pred == 0) & (y_true == 1)).sum()

    fpr = fp / (fp + tn + 1e-12)
    tpr = tp / (tp + fn + 1e-12)

    rows.append({
        "threshold": round(float(t), 2),
        "false_positive_rate": fpr,
        "true_positive_rate": tpr
    })



Save this data in a file "roc.tsv" with columns threshold, false_positive_rate and true_positive rate.

In [15]:
# YOUR CHANGES HERE

roc = pd.DataFrame(rows)
roc.to_csv("roc.tsv", sep="\t", index=False)
print(f"Saved roc.tsv with {len(roc)} rows.")
roc.head()


Saved roc.tsv with 99 rows.


,threshold,false_positive_rate,true_positive_rate
0,0.01,0.034420,0.996503
1,0.02,0.034420,0.996503
2,0.03,0.032609,0.996503
3,0.04,0.030797,0.996503
4,0.05,0.028986,0.994755


Submit "roc.tsv" in Gradescope.

## Part 7: Signup for Gemini API Key

Create a free Gemini API key at https://aistudio.google.com/app/api-keys.
You will need to do this with a personal Google account - it will not work with your BU Google account.
This will not incur any charges unless you configure billing information for the key.

You will be asked to start a Gemini free trial for week 11.
This will not incur any charges unless you exceed expected usage by an order of magnitude.


No submission needed.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 9: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

In [16]:


acknowledgements = """
Acknowledgements:
I completed this assignment on my own.

Libraries Used:
none

Generative AI Tools:
none except for part 7
"""

with open("acknowledgements.txt", "w") as f:
    f.write(acknowledgements)

print(" acknowledgements.txt ")



 acknowledgements.txt 
